In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
import nltk

from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0

from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('punkt')

# Getting the list of stopwords
stop_words = set(stopwords.words('english'))

def remove_noise(text):
    # Remove special characters and numbers
    #text = re.sub(r'[^a-zA-Z\s?]', '', str(text)) #keeps question marks
    text = re.sub(r'[^a-zA-Z\s]', '', str(text))
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def remove_emojis_and_links(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    link_pattern = re.compile(r'http\S+|www\S+')
    text = emoji_pattern.sub(r'', text)
    text = link_pattern.sub(r'', text)
    return text

def is_english(text):
    try:
        return detect(text) == 'en' 
    except:
        return False  # Return False if detection fails

# Applying the language detection function to filter only English records
# df = df[df['text'].apply(is_english)]

# normalization
df['cleaned_text'].str.lower()



# Function to remove stopwords
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Applying the function to the cleaned_text column
# df['cleaned_text'].apply(remove_stopwords)
# df['cleaned_text'].head()



# df['tokenized_text'] = df['cleaned_text'].apply(nltk.word_tokenize)
# df['tokenized_text'].head()

from nltk.stem import PorterStemmer

# Initialize the stemmer
stemmer = PorterStemmer()

# Function to stem the tokens
def stem_tokens(tokens):
    return [stemmer.stem(token) for token in tokens]

# Apply the stemming function to the tokenized_text column
df['stemmed_text'] = df['tokenized_text'].apply(stem_tokens)
df['stemmed_text'].head()

#text preprocessing function
def text_preprocessing(text):
    # Implement any preprocessing steps here, e.g., removing punctuation, lowercasing, etc.
    return text.lower()

# Load your dataset
# Assuming df['text'] contains the text and df['label'] the target
df = pd.DataFrame({
    'text': ["Sample issue text 1", "Sample issue text 2"],  # replace with actual text
    'label': ["question", "bug"]  # replace with actual labels
})

X = df['text']
y = df['label']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the pipeline
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(preprocessor=text_preprocessing)),  # Feature extraction with text preprocessing
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))  # Random Forest model
])

# Train the model
pipeline.fit(X_train, y_train)

# Evaluate the model
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))


In [ ]:
import re
import nltk
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from langdetect import detect, DetectorFactory
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

DetectorFactory.seed = 0

nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

# Define each preprocessing function

def remove_noise(text):
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', str(text))
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def remove_emojis_and_links(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags
                               "]+", flags=re.UNICODE)
    link_pattern = re.compile(r'http\S+|www\S+')
    text = emoji_pattern.sub(r'', text)
    text = link_pattern.sub(r'', text)
    return text

def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False  # Return False if detection fails

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

def stem_text(text):
    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

def text_preprocessing(text):
    # Check if text is in English
    if not is_english(text):
        return ""
    
    # Apply all preprocessing steps
    text = text.lower()                         # Normalization
    text = remove_noise(text)                   # Remove noise
    text = remove_emojis_and_links(text)        # Remove emojis and links
    text = remove_stopwords(text)               # Remove stopwords
    text = stem_text(text)                      # Apply stemming
    
    return text

# Build the pipeline with the consolidated preprocessing
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(preprocessor=text_preprocessing)),  # Feature extraction with text preprocessing
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))  # Random Forest model
])
"""
# Split data (assuming df['text'] and df['label'] exist)
X = df['text']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
pipeline.fit(X_train, y_train)

# Evaluate the model
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))
"""

In [1]:
import pandas as pd

# Load the dataset
df = pd.read_csv('dataset.csv')

# Select only the columns you need
df = df[['stemmed_text', 'issue_label']]

# Drop any rows with missing values in these columns, if necessary
df.dropna(subset=['stemmed_text', 'issue_label'], inplace=True)

# Separate features and labels
X = df['stemmed_text']  # Text column
y = df['issue_label']    # Label column

# Check the data to ensure it loaded correctly
print(df.head())
print(f"Features: {X.shape}, Labels: {y.shape}")


                                        stemmed_text issue_label
0  ['tilesrcrect', 'null', 'entitiesldtk', 'sampl...         bug
1  ['updat', 'blog', 'link', 'sef', 'site', 'desc...         bug
2  ['parser', 'can', 'not', 'properli', 'distingu...         bug
3  ['row', 'ad', 'snackbar', 'visibl', 'grid', 's...         bug
4  ['crash', 'tcpwriteonsocket', 'crash', 'flush'...         bug
Features: (59973,), Labels: (59973,)


In [ ]:
# Load the dataset
df = pd.read_csv('dataset.csv')

# Select only the columns you need
df = df[['stemmed_text', 'issue_label']]

# Drop any rows with missing values in these columns, if necessary
df.dropna(subset=['stemmed_text', 'issue_label'], inplace=True)

# Separate features and labels
X = df['stemmed_text']  # Text column
y = df['issue_label']   # Label column

# Split data into train, validation, and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.2, random_state=42, stratify=y_train_val
)

pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(preprocessor=text_preprocessing)),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'))
])

# Train the model on the training set
pipeline.fit(X_train, y_train)

# Evaluate the model on the validation set
y_val_pred = pipeline.predict(X_val)
print("Validation Set Classification Report:\n", classification_report(y_val, y_val_pred))

# Evaluate the model on the test set
y_test_pred = pipeline.predict(X_test)
print("Test Set Classification Report:\n", classification_report(y_test, y_test_pred))
